In [ ]:
import os, random, numpy as np, pandas as pd, torch, lightgbm as lgb
SEED = 42
np.random.seed(SEED); random.seed(SEED); torch.manual_seed(SEED)

import torch
from torch import nn
from PIL import Image

from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset

import os
import pandas as pd
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
DATA_DIR = "/kaggle/input/cassava-leaf-disease-classification"
df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class CassavaDataset(Dataset):
    def __init__(self, df, img_dir='train_images',transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir,self.df.iloc[idx]['image_id'])
        image = Image.open(img_path).convert('RGB')
        label = self.df.iloc[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image,label


train_dataset = CassavaDataset(df, img_dir='train_images', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [ ]:
class LFModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False

        num_features = self.backbone.fc.in_features

        self.backbone.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),

            nn.Linear(128, 5)
        )

    def forward(self, x):
        return self.backbone(x)

model = LFModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

In [ ]:
n_epochs = 15  # Reduce for Kaggle runtime limits

for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_dataloader)
    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}")

# Save model
torch.save(model.state_dict(), "/kaggle/working/model_low_lr.pth")

In [ ]:
import torch
from torch import nn
from PIL import Image

from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset

import os
import pandas as pd
import numpy as np

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
DATA_DIR = "/kaggle/input/cassava-leaf-disease-classification"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

model = LFModel().to(device)
model.load_state_dict(torch.load("/kaggle/input/cassava-model/model_low_lr.pth", map_location=device))

In [ ]:
class CassavaTestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_ids = os.listdir(img_dir)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_id

In [ ]:
model.eval()

test_dataset = CassavaTestDataset(
    img_dir=os.path.join(DATA_DIR, "test_images"),
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=32,num_workers=4,pin_memory=True)

predictions = []
image_ids = []

with torch.inference_mode():
    for images, img_ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(preds)
        image_ids.extend(img_ids)

In [ ]:
submission_df = pd.DataFrame({
    "image_id": image_ids,
    "label": predictions
})

submission_df.to_csv("/kaggle/working/submission.csv", index=False)